<b>Introduction/Business Problem:</b>
<br>Montreal is a city covering 431.5 km^2 (166.6 mi^2) with approx. 1.8M inhabitants residing in 21 boroughs.

The City of Saints is visited by over 2M people each year; making it Canada's most visited city. With so much to see and do, I'm going to need a cup of coffee!

With that in mind, I'm going to examine The City of 100 Steeples and see which neighborhoods will shelter and stimulate weary travelers.

<b>Data:</b> 
<br>I will be using zipcode for metropolitan Montreal that is available on wikipedia. I will also be using venue name, location, and category information on Foursquare to identify neighborhoods that have accomodations for travelers (hotels/hostels/inns/etc) and coffee shops.

In [179]:
#putting tools in the toolbox
import pandas as pd
import numpy as np
import requests as re
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import os
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip -q install folium
import folium
print('Done')

Done


In [180]:
#linking our postal code and neighborhood data
zipcode_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H"
source=requests.get(list_url).text
#print(source)

In [181]:
#looking for patterns in the source code
soup=BeautifulSoup(source)
#print(soup.prettify())

In [182]:
#seperating postal code and neighborhood data from the rest of the page
#codes=soup.find_all('td',attrs={'valign':'top'})
#for code in codes:
#    print(code)
code_body=soup.find('tbody')
#code_body

In [208]:
#making a list of postal codes
postal_codes=code_body.find_all('b')
#postal_codes
code_list=[]
for p in postal_codes[0:]:
    result=p.text.strip()
    code_list.append(result)
#(code_list)

In [184]:
len(code_list)

181

In [185]:
#making a list of neighborhoods
neighborhoods=code_body.find_all('span')
#print(neighborhoods)
ngh_list=[]
for n in neighborhoods[0:]:
    ngh_results=n.text.strip()
    ngh_list.append(ngh_results)
#print(ngh_list)

In [186]:
len(ngh_list)

181

In [187]:
#create data frame with postal codes and neighborhoods
mtl_ngh=pd.DataFrame({'Postalcode':code_list,'Neighborhood':ngh_list})
mtl_ngh.head()

,Postalcode,Neighborhood
0,"H0A,",Not assigned
1,H1A,Pointe-aux-Trembles
2,H2A,"Saint-Michel,East"
3,H3A,Downtown Montreal North(McGill University)
4,H4A,Notre-Dame-de-GrâceNortheast


In [188]:
#remove postal codes that are not assigned
mtl_ngh=mtl_ngh[mtl_ngh['Neighborhood']!='Not assigned']
print(mtl_ngh.head())
len(mtl_ngh)

  Postalcode                                Neighborhood
1        H1A                         Pointe-aux-Trembles
2        H2A                           Saint-Michel,East
3        H3A  Downtown Montreal North(McGill University)
4        H4A                Notre-Dame-de-GrâceNortheast
5        H5A                           Place Bonaventure


124

In [189]:
mtl_temp=mtl_ngh.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
mtl_temp=mtl_temp.reset_index(drop=False)
mtl_temp.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
mtl_temp=mtl_temp[mtl_temp['Neighborhood_joined']!='Reserved0H0: Santa Claus']
print(mtl_temp.head())
len(mtl_temp)

  Postalcode Neighborhood_joined
1        H0J           AnjouWest
2        H0K           AnjouEast
3        H0L        MercierNorth
4        H0M         MercierWest
5        H0N    MercierSoutheast


123

In [192]:
mtl_merge=pd.merge(mtl_ngh,mtl_temp,on='Postalcode')
mtl_merge.drop(['Neighborhood'],axis=1,inplace=True)
mtl_merge.drop_duplicates(inplace=True)
mtl_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
print(mtl_merge.head())
len(mtl_merge)

  Postalcode                                Neighborhood
0        H1A                         Pointe-aux-Trembles
1        H2A                           Saint-Michel,East
2        H3A  Downtown Montreal North(McGill University)
3        H4A                Notre-Dame-de-GrâceNortheast
4        H5A                           Place Bonaventure


123